## To classify the randomly generated PDs

In [6]:
import glob
import numpy as np
import random as rdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC

import warnings
warnings.filterwarnings('ignore')

# to read PBs generated by random PDs
pb_files = glob.glob('./testPB/PB4distribution030/*.txt')

pb_files = sorted(pb_files)
print(len(pb_files))

data = []

for i in range(len(pb_files)):
    data.append(np.loadtxt(pb_files[i]))
    
data = np.array(data)

print(data.shape)



classifiers = [
    KNeighborsClassifier(5),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    LogisticRegression(),
    LinearSVC(C=10),
    SVC(kernel = "rbf", C=10,gamma = 0.001)
]

clf_name = []

for clf in classifiers:
    clf_name.append(clf.__class__.__name__)

log_cols = ["Classifier", "Accuracy", "Variance"]
log  = pd.DataFrame(columns=log_cols)




b = 1;

acc_dict = {}


for i in range(0,b):
    # 5 classes
    y = (20 * [0] + 20 * [1] + 20 * [2] + 20 * [3] + 20 * [4])
    rdm.shuffle(y)
    y = np.array(y)

    train = np.c_[data, y]

    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

    X = train[:, :-1] # data
    Y = train[:, -1]  # label

    for train_index, test_index in sss.split(X, Y):
        X_train, X_test = X[train_index], X[test_index] # data
        Y_train, Y_test = Y[train_index], Y[test_index] # label

        for clf in classifiers:
            name = clf.__class__.__name__
            clf.fit(X_train, Y_train)
            train_predictions = clf.predict(X_test)
            acc = accuracy_score(Y_test, train_predictions)
            # the key of dictionary: (name,i)
            if (name,i) in acc_dict:
                acc_dict[name,i] += acc
            else:
                acc_dict[name,i] = acc
            
# The average accuracy of 100 tests
for clf in acc_dict:
    acc_dict[clf] = acc_dict[clf] / 10.0

acc_ave = {}
acc_var = {}
# To compute the average and variance
for clfn in clf_name:
    ave = 0
    var = 0
    for i in range(0,b):
        ave += acc_dict[clfn,i] 
    ave /= b
    for i in range(0,b):
        var += np.square(acc_dict[clfn,i]-ave)
    acc_ave[clfn] = ave
    acc_var[clfn] = var
    
for clf in acc_var:
    log_entry = pd.DataFrame([[clf, acc_ave[clf], acc_var[clf]]], columns=log_cols)
    log = log.append(log_entry)

                                                                                                                                                                                                                                                                                                                      
log

100
(100, 400)


,Classifier,Accuracy,Variance
0,KNeighborsClassifier,0.203333,0.0
0,RandomForestClassifier,0.183333,0.0
0,GradientBoostingClassifier,0.203333,0.0
0,LogisticRegression,0.240000,0.0
0,LinearSVC,0.233333,0.0
0,SVC,0.176667,0.0
